The basemap regridding tutorial that this is based upon is located at: http://earthpy.org/interpolation_between_grids_with_pyresample.html

In [37]:
# plotting stuff
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import cartopy.crs as crs

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import scipy
import xarray as xr
import netCDF4
from wrf import (to_np, getvar, ALL_TIMES, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
import pyresample.bilinear as prsbilinear
import optwrf as ow
from optwrf import runwrf as rw

In [2]:
param_ids = [10, 1, 1, 2, 2, 3, 2]
start_date = 'Feb 7  2011'
end_date = 'Feb 8 2011'
wrf_sim = rw.WRFModel(param_ids, start_date, end_date, 
                      setup_yaml='/Users/swardy9230/Box Sync/01_Research/01_Renewable_Analysis/WRF_Solar_and_Wind/met4ene/optwrf/optwrf/data/mac_dirpath.yml') 

Forecast starting on: 2011-02-07 00:00:00
Forecast ending on: 2011-02-08 00:00:00


In [9]:
# Process the ERA5 data
wrf_sim.process_era5_data()

In [10]:
# Find the path to optwrf
optwrf_path = os.path.dirname(os.path.abspath(inspect.getfile(ow)))

In [11]:
in_yr = 2011
in_mo = 2
wrfdir = os.path.join(optwrf_path, 'data/')
eradir = wrf_sim.DIR_ERA5_ROOT

In [12]:
# WRF file containing source grid
wrffile = 'wrfout_processed_d01.nc'
try:
    wrfdata = xr.open_dataset(wrfdir + wrffile)
except FileNotFoundError:
    print(f'The wrfout file {wrfdir + wrffile} does not exist. Check that your path.')

# Get wrf variable(s) to regrid
# wrf_lat = wrfdata.XLAT
# wrf_lon = wrfdata.XLONG
# wrfdata = wrfdata.rename({'XLONG': 'lon', 'XLAT': 'lat'})

# Read in and convert GHI from W m-2 to kW m-2
ghi = wrfdata.ghi
ghi = ghi/1000

# Read in WPD, convert from W m-2 to kW m-2
wpd = wrfdata.wpd
wpd = wpd/1000

wrfdata

<xarray.Dataset>
Dimensions:         (Time: 24, bottom_top: 35, south_north: 191, west_east: 191)
Coordinates:
  * Time            (Time) datetime64[ns] 2011-02-07 ... 2011-02-07T23:00:00
    lon             (south_north, west_east) float32 ...
    lat             (south_north, west_east) float32 ...
    wspd_wdir       object ...
    XLONG           (south_north, west_east) float32 ...
    XLAT            (south_north, west_east) float32 ...
    XTIME           (Time) float32 ...
    level           int64 ...
Dimensions without coordinates: bottom_top, south_north, west_east
Data variables:
    Times           (Time) datetime64[ns] ...
    temp_air        (Time, south_north, west_east) float32 ...
    wind_speed_u    (Time, south_north, west_east) float32 ...
    wind_speed_v    (Time, south_north, west_east) float32 ...
    cloud_fraction  (Time, bottom_top, south_north, west_east) float32 ...
    cos_zenith      (Time, south_north, west_east) float32 ...
    dni             (Time, south_north, west_east) float32 ...
    dhi             (Time, south_north, west_east) float32 ...
    ghi             (Time, south_north, west_east) float32 0.0 0.0 ... 0.0 0.0
    wind_speed10    (Time, south_north, west_east) float32 ...
    wind_speed100   (Time, south_north, west_east) float32 ...
    wpd             (Time, south_north, west_east) float32 29141.006 ... 7670...

In [13]:
# ERA data file(s)
erafile = f'ERA5_EastUS_WPD-GHI_{str(in_yr).zfill(4)}-{str(in_mo).zfill(2)}.nc'
try:
    eradata = xr.open_dataset(eradir + erafile)
except FileNotFoundError:
    print(f'The wrfout file {eradir + erafile} does not exist. Check that your path.')

# Get variables to compare with regridded WRF variables.
# era_lat = eradata.latitude
# era_lon = eradata.longitude
eradata = eradata.rename({'longitude': 'lon', 'latitude': 'lat'})

# Read in ERA_GHI, convert from W m-2 to kW m-2
era_ghi = eradata.GHI
era_ghi = era_ghi / 1000

# Read in ERA_WPD, convert from W m-2 to kW m-2
era_wpd = eradata.WPD
era_wpd = era_wpd / 1000

# Write these back to the xarray dataset
eradata['ghi'] = era_ghi
eradata['wpd'] = era_wpd

eradata = eradata.sortby('lat', ascending=True)

eradata

<xarray.Dataset>
Dimensions:   (Time: 672, lat: 97, lon: 129, time: 672)
Coordinates:
  * lat       (lat) float64 28.0 28.25 28.5 28.75 29.0 ... 51.25 51.5 51.75 52.0
  * lon       (lon) float64 264.0 264.2 264.5 264.8 ... 295.2 295.5 295.8 296.0
  * time      (time) datetime64[ns] 2011-02-01 ... 2011-02-28T23:00:00
  * Time      (Time) datetime64[ns] 2011-02-01 ... 2011-02-28T23:00:00
Data variables:
    VAR_100U  (time, lat, lon) float32 -6.3804474 -6.064041 ... -3.508957
    VAR_100V  (time, lat, lon) float32 6.174362 6.1294403 ... 8.91597 8.91597
    WPD       (time, lat, lon) float32 349970.1 320499.16 ... 432805.34 439831.0
    SSRD      (Time, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    GHI       (Time, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ghi       (Time, lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    wpd       (time, lat, lon) float32 349.9701 320.49915 ... 432.80536 439.831

In [33]:
# GridDefinition() requires that lons and lats have the same shape
lon2d, lat2d = np.meshgrid(eradata.lon - 180, eradata.lat)
era_def = prs.geometry.SwathDefinition(lons=lon2d, lats=lat2d)
era_def

In [23]:
wrf_def = prs.geometry.SwathDefinition(lons=wrfdata.lon, lats=wrfdata.lat)
wrf_def

In [34]:
wrf_ghi_regrid = prs.kd_tree.resample_nearest(wrf_def, wrfdata.ghi, \
        era_def, radius_of_influence=500000, fill_value=None)

AttributeError: 'DataArray' object has no attribute 'ravel'

In [38]:
# Do the regridding
wrf_ghi_regrid = prsbilinear.xarr.XArrayResamplerBilinear(wrfdata.ghi, wrf_def, era_def, radius=50e3, fill_value=None)
wrf_ghi_regrid

AttributeError: module 'pyresample.bilinear' has no attribute 'xarr'

In [ ]:
# Add the regridded variables to the WRF xarray dataset
wrfdata['ghi_regrid'] = wrf_ghi_regrid
wrfdata['wpd_regrid'] = wrf_wpd_regrid

In [ ]:
ghi2d = wrfdata.ghi.isel(Time=14)
ghi2d.plot()

In [ ]:
ghi2d = wrfdata.ghi_regrid.isel(Time=14)
ghi2d.plot()

In [ ]:
wrf_wpd_regrid
wpd2d = wrfdata.wpd_regrid.isel(Time=14)
wpd2d.plot()

In [ ]:
# Now test the function that does the same as the above in optwrf.runwrf
wrfdata_2, eradata_2 = rw.wrf_era5_regrid_xesmf(in_yr, in_mo, wrfdir, eradir)

In [ ]:
ghi2d = wrfdata_2.ghi_regrid.isel(Time=14)
ghi2d.plot()

In [ ]:
wrf_wpd_regrid
wpd2d = wrfdata_2.wpd_regrid.isel(Time=14)
wpd2d.plot()

In [ ]:
eradata_2

In [ ]:
wrfdata_2

In [ ]:
ghi_error = abs(wrfdata_2.ghi_regrid - eradata_2.ghi)
ghi_error[0,0,:50]

In [ ]:
# To get around xrray nan bug, fill all nan with -1 (since we took the abs(), all the values in our GHI error should be >= 0)
ghi_error_nonan = ghi_error.fillna(-1)
ghi_error_nonan[0,0,:50]

In [ ]:
total_ghi_error = ghi_error_nonan.sum(dim='Time')
total_ghi_error = total_ghi_error.where(total_ghi_error > 0)
total_ghi_error.plot()

In [ ]:
# Manually calculate the sum to make sure that they are the same
for ii in range (0, 24):
    if ii == 0:
        total_ghi_error_man = ghi_error.isel(Time=ii) 
    else:
        total_ghi_error_man += ghi_error.isel(Time=ii) 
total_ghi_error_man.plot()

In [ ]:
# Compare the manual and hacky bug fix. (All should be nan or 0)
diff = total_ghi_error_man - total_ghi_error_man
diff.plot()

In [ ]:
print(f'The difference between the two is: {diff.sum().values}')
print(f'The total GHI error as calculated by the hacky fix: {float(total_ghi_error.sum().values)}')
print(f'The total GHI error as calculated manually: {float(total_ghi_error_man.sum().values)}')

In [ ]:
# Get the total error now using the function in optwrf.runwrf
wrfdata_2 = rw.wrf_era5_error(wrfdata_2, eradata_2)
wrfdata_2

In [ ]:
error = [0, float(wrfdata_2['total_ghi_error'].sum().values), float(wrfdata_2['total_wpd_error'].sum().values)]
error

In [ ]:
# Annual mean daylight fraction 
daylight_factors = []
for jday in range(1, 366):
    daylight_factors.append(ow.helper_functions.daylight_frac(jday))

In [ ]:
fig, ax = plt.subplots()
ax.plot(daylight_factors)

In [ ]:
sum(daylight_factors) / len(daylight_factors)

The following function makes use of the NCAR geocat.comp module. Unfortunately, the function I was looking to use, rcm2rgrid, does not yet exist... One day

In [ ]:
def wrf_era5_regrid_geocat(in_yr, in_mo, wrfdir='./', eradir='/share/mzhang/jas983/wrf_data/data/ERA5/'):
    """

    :param in_yr:
    :param in_mo:
    :param wrfdir:
    :param eradir:
    :return:

    """
    # WRF file containing source grid
    wrffile = 'wrfout_processed_d01.nc'
    try:
        wrfdata = xr.open_dataset(wrfdir + wrffile)
    except FileNotFoundError:
        print(f'The wrfout file {wrfdir + wrffile} does not exist. Check that your path.')
        wrfdata = None
        eradata = None
        return wrfdata, eradata

    # Get wrf variable(s) to regrid
    wrf_lat = wrfdata.XLAT
    wrf_lon = wrfdata.XLONG

    # Read in and convert GHI from W m-2 to kW m-2
    ghi = wrfdata.ghi
    ghi = ghi / 1000

    # Read in WPD, convert from W m-2 to kW m-2
    wpd = wrfdata.wpd
    wpd = wpd / 1000

    # ERA data file(s)
    erafile = f'ERA5_EastUS_WPD-GHI_{str(in_yr).zfill(4)}-{str(in_mo).zfill(2)}.nc'
    try:
        eradata = xr.open_dataset(eradir + erafile)
    except FileNotFoundError:
        print(f'The wrfout file {eradir + erafile} does not exist. Check that your path.')
        eradata = None
        return wrfdata, eradata

    # Get variables to compare with regridded WRF variables.
    era_lat = eradata.latitude
    era_lon = eradata.longitude

    # Read in ERA_GHI, convert from W m-2 to kW m-2
    era_ghi = eradata.GHI
    era_ghi = era_ghi / 1000

    # Read in ERA_WPD, convert from W m-2 to kW m-2
    era_wpd = eradata.WPD
    era_wpd = era_wpd / 1000

    # Write these back to the xarray dataset
    eradata['ghi'] = era_ghi
    eradata['wpd'] = era_wpd

    # Do the regridding
    wrf_ghi_regrid = geocat.comp.rcm2rgrid(wrf_lat, wrf_lon, ghi, era_lat, era_lon)
    wrf_wpd_regrid = geocat.comp.rcm2rgrid(wrf_lat, wrf_lon, wpd, era_lat, era_lon)

    # Add the regridded variables to the WRF xarray dataset
    wrfdata['ghi_regrid'] = wrf_ghi_regrid
    wrfdata['wpd_regrid'] = wrf_wpd_regrid

    return wrfdata, eradata
